# Demo of watershed Algorithm, segmenting by clicking

Run all the cells and the final application cell. This opens 2 windows, one with the actual image and the other on which the "seeds" of the Watershed algorithm will be created. 
Use 0-9 to select a color, then click at any point on the actual-image window. You will see a segment with that point as a seed and your chosen colour

Note: To start over, please run all cells again

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
road = cv2.imread('./DATA/road_image.jpg')
road_copy = road.copy()

Note: We use color correction for the sole purpose of Matplotlib viewing. But remember that for application showing, we use cv2.imshow('name', image) instead of plt.show(). So, in case of application viewing (popup types), there is no need to do color corrections

### Create the marker image, 2D only

In [3]:
print(road.shape)
print(road.shape[:2])
marker_image = np.zeros(road.shape[:2], dtype = np.int32)

(600, 800, 3)
(600, 800)


### Create the segments image, 3D

In [4]:
segments = np.zeros(road.shape, dtype = np.uint8)
segments.shape

(600, 800, 3)

### Qualitative color mapping:

The idea is to press a number from 0 - 9 which selects a particular color. For this we use the qualitative color mapping, cm.tab10

In [5]:
from matplotlib import cm

#### A function that creates and sends back the color tuple, when required

In [6]:
def create_rgb(i):
    return tuple(np.array(cm.tab10(i)[:3]) * 255)

In [7]:
colors = []
for i in range(10):
    colors.append(create_rgb(i))

print(colors)

# As we can see below, 10 colors are added to colors[]

[(31.0, 119.0, 180.0), (255.0, 127.0, 14.0), (44.0, 160.0, 44.0), (214.0, 39.0, 40.0), (148.0, 103.0, 189.0), (140.0, 86.0, 75.0), (227.0, 119.0, 194.0), (127.0, 127.0, 127.0), (188.0, 189.0, 34.0), (23.0, 190.0, 207.0)]


## Application:

In [8]:
# GLOBAL VARIBLES:
current_marker = 1
marks_updated = False

# CALLBACK FUNCTION:
def mouse_callback(event, x, y, flags, param):
    global marks_updated
    
    if(event == cv2.EVENT_LBUTTONDOWN):
        
        # MARKERS PASSED TO WATERSHED
        cv2.circle(marker_image, center = (x, y), radius = 5, color = current_marker, thickness = -1)
        
        # USER SEES:
        cv2.circle(road_copy, center = (x, y), radius = 5, color = colors[current_marker], thickness = -1)
        
        marks_updated = True
        
        
# CONNECTING THE CALLBACK:
cv2.namedWindow(winname = 'Road Image')
cv2.setMouseCallback('Road Image', mouse_callback)


while True:
    cv2.imshow('Watershed Segments', segments)
    cv2.imshow('Road Image', road_copy)
    
    
    k = cv2.waitKey(1)
    
    if(k == 27):
        break
    
    # CLEAR ALL COLORS (RESET) ON PRESSING "c"
    elif(k == ord('c')):
        road_copy = road.copy()
        marker_image = np.zeros(road.shape[:2], np.int32)
        segments = np.zeros(road.shape, np.uint8)
    
    
    # UPDATE COLOR CHOICE: (ON NUMBER PRESS 0-9), choose that particular color
    elif(k > 0 and chr(k).isdigit()):            # chr: Convert ASCII value to actual digit, still a string,
        current_marker = int(chr(k))             #      which is why we take int(chr(k))
    
    
    # UPDATE MARKINGS:
    if(marks_updated):
        marker_image_copy = marker_image.copy()
        cv2.watershed(road, marker_image_copy)
        
        segments = np.zeros(road.shape, dtype = np.uint8)
        
        for i in range(len(colors)):
            # COLOR THE SEGMENTS
            segments[marker_image_copy == i] = colors[i]
        marks_updated = False


cv2.destroyAllWindows()